In [7]:
import numpy as np
from keras.layers.core import Dense, Activation, Flatten, Dropout, Lambda, Reshape
from keras.layers.advanced_activations import ELU
from keras.layers.convolutional import Convolution2D, Cropping2D
from keras.layers.pooling import MaxPooling2D
from keras.models import Sequential
import csv
import cv2
import math
import matplotlib.image as mpimg
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [ ]:
def load_image(path):
    image = mpimg.imread(path)
    #image = cv2.resize(image,(66,200),interpolation=cv2.INTER_AREA)   
    #image = image[50:-18, :]
    return image

def flip_image(image):
    flipped_image = cv2.flip(image, 1)
    return flipped_image

def resize(image):
    return tf.image.resize_images(image, (66, 200))
 

def generate_training_data(BATCH_SIZE, X_train, y_train):
    while True:
        num_examples = len(y_train)
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            images = list()
            labels = list()
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            for k in range(len(batch_x)):
                image = load_image(batch_x[k])
                steering = float(batch_y[k])
                flip_option = np.random.randint(2)
                images.append(image)
                labels.append(steering)              
                if flip_option==1:
                    flipped_steering = steering * -1.0
                    images.append(flip_image(image))
                    labels.append(flipped_steering)
                    
            #print('Lable Size', len(labels))
            yield shuffle(np.array(images, dtype=np.float64),np.array(labels, dtype=np.float64))


def normalize(X):
    return X/255.0 - 0.5

def load_driving_log(path):
    f = open(path)
    reader = csv.reader(f)
    X_train = []
    y_train = []
    i = 0
    LOAD_SIZE = math.inf
    for line in reader:
        slc = np.random.randint(3)
        X_train.append(line[0].strip())
        y_train.append(float(line[3].strip()))
        X_train.append(line[1].strip())
        y_train.append(float(line[3].strip()) + 0.2)
        X_train.append(line[2].strip())
        y_train.append(float(line[3].strip()) - 0.2)
        i = i + 1
        if(i > LOAD_SIZE):
            break;
        
        
    f.close()
    return X_train, y_train

In [ ]:
drive_log = 'driving_log.csv'
X_train,y_train = load_driving_log(drive_log)
X_train, y_train = shuffle(X_train, y_train)
print('Total Lines Loaded ', len(y_train))
X_train, X_cv, y_train, y_cv = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
num_examples = len(y_train)
print('Total Training Loaded ', len(y_train))
print('Total CV Loaded ', len(y_cv))

model = Sequential()
#model.add(Lambda(resize, input_shape=(160,320,3), output_shape=(66, 200, 3)))
model.add(Cropping2D(cropping=((60, 20), (0, 0)),input_shape=(160, 320, 3)))
#model.add(Cropping2D(cropping=((10, 5), (0, 0))))
#model.add(Lambda(Reshape((66, 200))))
model.add(Lambda(normalize))
model.add(Convolution2D(24, 5, 5, subsample=(2, 2), activation='relu'))
#model.add(MaxPooling2D(pool_size=(3, 3), strides=None, border_mode='valid'))
#model.add(Dropout(0.2))
#model.add(Convolution2D(24, 5, 5, subsample=(2, 2), border_mode="same", input_shape=(160, 320, 3)))
#model.add(Activation('relu'))
#model.add(ELU())
model.add(Convolution2D(36, 5, 5, subsample=(2, 2), activation='relu'))
model.add(Dropout(0.2))
#model.add(MaxPooling2D(pool_size=(3, 3), strides=None, border_mode='valid'))
#model.add(Dropout(0.2))
#model.add(Activation('relu'))
#model.add(ELU())
model.add(Convolution2D(48, 5, 5, subsample=(2, 2), activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2), strides=None, border_mode='valid'))
#model.add(Dropout(0.5))
#model.add(Activation('relu'))
#model.add(ELU())
model.add(Convolution2D(64, 3, 3, activation='relu'))
#model.add(Convolution2D(64, 3, 3, activation='relu'))
#model.add(MaxPooling2D(pool_size=(3, 3), strides=None, border_mode='valid'))
model.add(Dropout(0.2))
#model.add(ELU())
model.add(Flatten())
#model.add(Dropout(.2))
#model.add(ELU())
#model.add(Activation('relu'))
model.add(Dense(1164))
#model.add(Dropout(.2))
#model.add(Activation('relu'))
model.add(Dense(100))
#model.add(Dropout(.2))
#model.add(Activation('relu'))
model.add(Dense(50))
#model.add(Dropout(.2))
#model.add(Activation('relu'))
model.add(Dense(10))#model.add(Dropout(.2))
#model.add(Activation('relu'))
model.add(Dense(1))
model.summary()

# TODO: Compile and train the model
model.compile(optimizer="adam", loss="mse", metrics=['accuracy'])

BATCH_SIZE = 128
EPOCHS = 3
history=model.fit_generator(generator=generate_training_data(BATCH_SIZE=BATCH_SIZE, X_train=X_train, y_train=y_train),samples_per_epoch=num_examples*2, nb_epoch=EPOCHS, validation_data=generate_training_data(BATCH_SIZE=BATCH_SIZE, X_train=X_cv, y_train=y_cv), nb_val_samples=len(y_cv), verbose=1)
#history=model.fit_generator(generator=generate_training_data(BATCH_SIZE=BATCH_SIZE, X_train=X_train, y_train=y_train),samples_per_epoch=num_examples*2, nb_epoch=EPOCHS, verbose=2)
print(history.history)
model.save('model.h5')
print('Accuracy is ' + str(history.history['acc'][-1]))
#print('Validation accuracy is ' + str(history.history['val_acc'][-1]))

Total Lines Loaded  77433
Total Training Loaded  61946
Total CV Loaded  15487
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
cropping2d_2 (Cropping2D)        (None, 80, 320, 3)    0           cropping2d_input_2[0][0]         
____________________________________________________________________________________________________
lambda_2 (Lambda)                (None, 80, 320, 3)    0           cropping2d_2[0][0]               
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 38, 158, 24)   1824        lambda_2[0][0]                   
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 17, 77, 36)    21636       convolution2d_1[0][0]            
_____________

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


123977/123892 [==============================] - 411s - loss: 0.0452 - acc: 0.2200 - val_loss: 0.0339 - val_acc: 0.2197
Epoch 2/3
123897/123892 [==============================] - 341s - loss: 0.0327 - acc: 0.2208 - val_loss: 0.0261 - val_acc: 0.2199